# Evaluate Sum of Last K

You are given a list of data points, where each data point consists of: a list of tags, a list of timestamps (representing when the values were recorded), and a list of values (corresponding to each timestamp).


Your task is to:

1. Filter the data points based on a given tag.

2. Sort the filtered data points in ascending order by their timestamps.

3. Calculate the sum of the last K values from this sorted list.



In [2]:
from typing import Dict, List

def evaluate_sum_of_last_k(data, k, tag):
    tags = data["tag"]
    values = data["value"]
    timestamps = data["timestamp"]
    if tag in tags:
        pairs = list(zip(timestamps, values))
        pairs.sort(key=lambda x: x[0]) # sort by timestamp in ascending order
        pairs = pairs[-k:]
        return sum(pair[1] for pair in pairs)
    return 0  # If the tag is not found

# Example input
data = {
    "tag": ["env:prod", "prod1"],
    "timestamp": [1, 10, 3, 100, 2],
    "value": [-1, 10, -10, 100, 2]
}

k = 2
tag = "prod1"

# Call the function with the provided input
result = evaluate_sum_of_last_k(data, k, tag)
print(result)  # Output: 110

110


# 2 Coin Change

You are given an integer array of available coin denominations (coins) and an integer amount (amount). Your goal is to determine the minimum number of coins needed to make up that amount. If it's impossible to make the amount with the given coins, return -1.

In [6]:
# Greedy approach (only works for some coin denominations)
def coin_change_greedy(coins, amount):
    coins.sort(reverse=True)  # Sort coins in descending order
    count = 0
    for coin in coins:
        if amount == 0:
            break
        count += amount // coin  
        amount %= coin           
    return count if amount == 0 else -1  

# Dynamic Programming (DP) approach (guarantees the optimal solution)
def coin_change_dp(coins, amount):
    if amount == 0:
        return 0
    dp = [float("inf")] * (amount+1)
    dp[0] = 0 # to have $i money, minimum coins needed
    for i in range(1, amount+1):
        for coin in coins:
            if coin <= i:
                dp[i] = min(dp[i], dp[i-coin]+1)
    return dp[amount] if dp[amount] != float("inf") else -1

# Test both methods with the example [1, 3, 4] coins for amount 6
coins = [1, 3, 4]
amount = 6

# Test Greedy approach
result_greedy = coin_change_greedy(coins, amount) # 4, 1, 1

# Test DP approach
result_dp = coin_change_dp(coins, amount) # 3, 3

print(result_greedy, result_dp)

3 2


# 3 Calculate the File Sizes in a Folder

You are given a directory (folder) containing files and subfolders. Your task is to traverse through the directory, identify all the files, and calculate the sum of sizes of each file. 

In [12]:
import os

def calculate_folder_size(folder_path):
    total_size = 0
    
    # Loop over all the files and subfolders in the current folder
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        
        # If it's a file, add its size
        if os.path.isfile(item_path):
            total_size += os.path.getsize(item_path)

        # If it's a folder, recursively calculate the size of the subfolder
        elif os.path.isdir(item_path):
            total_size += calculate_folder_size_recursive(item_path)
    
    return total_size

# Example usage
folder_path = "/Users/cathzzr2/Desktop/SRI2024"
total_size = calculate_folder_size(folder_path)
print(str(total_size) + " bytes")


6001904 bytes


In [ ]:
# File/Directory Interface is given
# Use isinstance() function
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size

    def get_size(self):
        return self.size

class Directory:
    def __init__(self, name):
        self.name = name
        self.children = []

    def add_child(self, child):
        self.children.append(child)

    def get_size(self):
        total_size = 0
        for child in self.children:
            total_size += child.get_size()
        return total_size

class Solution:
    def calc_size_sum(self, directory) -> int:
        size = 0
        for item in directory.children:
            if isinstance(item, File):
                size += item.get_size()
            elif isinstance(item, Directory):
                self.calc_size_sum(item)
        return size


# 4 String Match

You are given two strings: a regular string and a compressed string. Your task is to determine if the compressed string matches the regular string. The compressed string may contain numeric values that represent any sequence of letters in the regular string.


Example 1:

datadog and d3dog -> true

Explanation: The 3 in d3dog means that three letters ("ata") in "datadog" are skipped, making the strings match.

Example 2:

datadog and d2dog -> false

Explanation: The 2 in d2dog skips two letters, but this does not allow the strings to match.

In [7]:
class Solution:
    def is_match(self, regular: str, compressed: str) -> bool:
        regular_idx, compressed_idx = 0, 0
        while regular_idx < len(regular) and compressed_idx < len(compressed):
            if compressed[compressed_idx].isdigit():
                skip_num = 0
                while compressed[compressed_idx].isdigit():
                    skip_num = skip_num * 10 + int(compressed[compressed_idx])
                    compressed_idx += 1
                regular_idx += skip_num
            else:
                if regular[regular_idx] != compressed[compressed_idx]:
                    return False
                regular_idx += 1
                compressed_idx += 1
        return True

# test
solution = Solution()
print(solution.is_match("dataaaaaaaaaadog", "d12dog"))  

True


# 5 String Match Follow-up

Based on #4, add a new condition: the compressed string can contain range patterns like {n,m}, meaning the string can skip between n and m letters. For example, d{1,3}dog would match d1dog, d2dog, or d3dog. Also, special characters like ^ might be introduced to represent a skip pattern, such as d^4dog, which can be interpreted the same way as d4dog.

In [9]:
class Solution:
    def is_match(self, regular: str, compressed: str) -> bool:
        regular_idx, compressed_idx = 0, 0
        
        while regular_idx < len(regular) and compressed_idx < len(compressed):
            if compressed[compressed_idx].isdigit():
                # Handle the case where compressed string has numbers (single skip)
                skip_num = 0
                while compressed_idx < len(compressed) and compressed[compressed_idx].isdigit():
                    skip_num = skip_num * 10 + int(compressed[compressed_idx])
                    compressed_idx += 1
                regular_idx += skip_num

            elif compressed[compressed_idx] == '^':
                compressed_idx += 1  # skip '^'

            elif compressed[compressed_idx] == '{':
                # Handle range pattern {n,m}
                compressed_idx += 1  # skip '{'
                range_start = 0
                range_end = 0
                
                # Parse the start of the range
                while compressed_idx < len(compressed) and compressed[compressed_idx].isdigit():
                    range_start = range_start * 10 + int(compressed[compressed_idx])
                    compressed_idx += 1
                
                if compressed[compressed_idx] == ',':
                    compressed_idx += 1  # skip ','
                
                # Parse the end of the range
                while compressed_idx < len(compressed) and compressed[compressed_idx].isdigit():
                    range_end = range_end * 10 + int(compressed[compressed_idx])
                    compressed_idx += 1
                
                if compressed[compressed_idx] == '}':
                    compressed_idx += 1  # skip '}'
                
                # Try skipping between range_start and range_end and check for match
                match_found = False
                for skip_count in range(range_start, range_end + 1):
                    if self.is_match(regular[regular_idx + skip_count:], compressed[compressed_idx:]):
                        match_found = True
                        break
                if not match_found:
                    return False
                regular_idx += skip_count
            else:
                # Compare the characters if not a digit or range pattern
                if regular[regular_idx] != compressed[compressed_idx]:
                    return False
                regular_idx += 1
                compressed_idx += 1
        
        # Check if both strings have been fully processed
        return regular_idx == len(regular) and compressed_idx == len(compressed)

# Test cases
solution = Solution()
print(solution.is_match("dataaaaaaaaaadog", "d12dog"))  # True
print(solution.is_match("datadog", "d{1,3}dog"))  # True
print(solution.is_match("datadog", "d2dog"))  # False
print(solution.is_match("datadog", "d3^dog"))  # True

True
True
False
True
